# Does the three strikes law deter crime? ###

### A time series analysis using state-level panel data in Python

In [101]:
import pandas as pd
import numpy as np

import statsmodels as sm
from statsmodels.formula.api import ols, mixedlm

### Grab the data

In [102]:
crime = pd.read_excel('crimedata.xlsx',sheet_name='Regression Data',skiprows=2).drop(columns='Unnamed: 0')

In [103]:
crime.head()

,Year,CR-Alab,CR-Alas,CR-Ari,CR-Ark,CR-Cal,CR-Col,CR-Con,CR-Del,CR-Dis,...,bud-SouD,bud-Ten,bud-Tex,bud-Uta,bud-Ver,bud-Vir,bud-Was,bud-Wes,bud-Wis,bud-Wyo
0,1984,3470.9,5493.6,5983.3,3046.1,5705.0,6013.3,4235.0,4571.1,6869.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1985,3484.6,5295.2,6513.7,3237.5,5752.7,6448.0,4302.8,4528.0,6374.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1986,3730.3,5675.5,6663.1,3529.9,5842.3,6508.3,4403.0,4404.6,6834.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1987,3892.2,4922.1,6576.0,3833.2,5588.4,5983.8,4576.7,4508.1,6841.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1988,4003.1,4398.8,6861.2,3796.9,5705.7,5705.7,4642.2,4347.4,7993.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Reformat dataframe

In [104]:
crime2 = pd.melt(crime,id_vars=['Year'],value_vars=crime.columns[1:])
crime2['metric'], crime2['state'] = crime2['variable'].str.split('-', 1).str
crime2 = crime2.drop(columns='variable')
crime2['state'] = crime2['state'].str.strip()


In [105]:
#crime3 = crime2.pivot(columns = 'metric', values = 'value')
crime3 = pd.pivot_table(crime2, values='value', index=['Year', 'state'],columns=['metric'], aggfunc=np.sum)
crime3.rename(columns={'3X':'Xlaw'}, inplace=True)

In [106]:
crime3['year'] = crime3.index.get_level_values(0)
crime3['State'] = crime3.index.get_level_values(1)

crime3.reset_index(level=0, inplace=True)
crime3.head()

metric,Year,Xlaw,AC,CR,HHI,IR,LPF,UR,YM,bud,year,State
state,,,,,,,,,,,,
Alab,1984,0.0,1.90,3470.9,34036.516854,256.0,188.819253,10.2,4251.325843,0.0,1984,Alab
Alas,1984,0.0,3.86,5493.6,63621.348315,253.0,201.188119,9.9,4145.586068,0.0,1984,Alas
Ari,1984,0.0,3.08,5983.3,42127.808989,247.0,235.709635,5.0,3992.674614,0.0,1984,Ari
Ark,1984,0.0,1.78,3046.1,30819.662921,190.0,160.656436,8.3,4103.029002,0.0,1984,Ark
Cal,1984,0.0,3.19,5705.0,49721.629213,162.0,224.686955,7.8,3955.264719,0.0,1984,Cal


### 1st: Run Prelim Pooled Least Squares

In [107]:
results = ols('CR ~ UR + HHI + LPF + YM + IR + AC + Xlaw', data=crime3).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     CR   R-squared:                       0.380
Model:                            OLS   Adj. R-squared:                  0.376
Method:                 Least Squares   F-statistic:                     97.46
Date:                Tue, 15 Jan 2019   Prob (F-statistic):          5.45e-111
Time:                        15:57:57   Log-Likelihood:                -9275.0
No. Observations:                1122   AIC:                         1.857e+04
Df Residuals:                    1114   BIC:                         1.861e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1572.6030    481.608      3.265      0.001     627.641    2517.565
UR           106.8194     18.424      5.798      0.000      70.669     142.970
HHI            0.0081      0.004      1.918      0.055      -0.000       0.016
LPF            2.1955      0.379      5.801      0.000       1.453       2.938
YM             0.0203      0.088      0.231      0.817      -0.152       0.192
IR             2.5602      0.161     15.927      0.000       2.245       2.876
AC           222.7118     55.610      4.005      0.000     113.600     331.824
Xlaw          87.3809     68.949      1.267      0.205     -47.903     222.664
==============================================================================
Omnibus:                       34.904   Durbin-Watson:                   2.048
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               37.360
Skew:                           0.438   Prob(JB):                     7.72e-09
Kurtosis:                       3.174   Cond. No.                     8.34e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.34e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 2nd: Fixed Effects for States

In [109]:
results = mixedlm('CR ~ UR + HHI + LPF + YM + IR + AC + Xlaw', data=crime3, groups=crime3['State']).fit()
results.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
           Mixed Linear Model Regression Results
============================================================
Model:             MixedLM  Dependent Variable:  CR         
No. Observations:  1122     Method:              REML       
No. Groups:        51       Scale:               203438.6561
Min. group size:   22       Likelihood:          -8551.3529 
Max. group size:   22       Converged:           Yes        
Mean group size:   22.0                                     
------------------------------------------------------------
            Coef.    Std.Err.   z    P>|z|  [0.025   0.975] 
------------------------------------------------------------
Intercept   3169.996  442.814  7.159 0.000 2302.096 4037.896
UR            14.103   12.646  1.115 0.265  -10.683   38.889
HHI           -0.010    0.005 -2.017 0.044   -0.020   -0.000
LPF            0.196    0.664  0.296 0.768   -1.106    1.498
YM            -0.314    0.066 -4.786 0.000   -0.443   -0.185
IR             1.915    0.086 22.165 0.000    1.746    2.085
AC           957.341   77.430 12.364 0.000  805.582 1109.100
Xlaw        -264.986   48.021 -5.518 0.000 -359.106 -170.865
Group Var 925752.031  431.692                               
============================================================

"""